In [46]:
from Signal_generator.generate_signal import generate_X_matrix
from Algorithmes.beamforming import beamforming_method
from Signal_generator.generate_signal import generate_A_matrix
from Signal_generator.generate_signal import generate_noise
from Signal_generator.generate_signal import generate_R_hat
from Signal_generator.generate_signal import generate_R_hat_with_phase
from Algorithmes.music import music_method
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split 
import numpy as np
import torch
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [47]:
nbSources = 2 # Nombre de sources
nbSensors = 10 # Nombre de capteurs
nbTimePoints = 100 # Nombre de points temporels
signal_noise_ratio = 3 # Rapport signal sur bruit en décibels. Si 'False', cela revient à une absence totale de bruit.
L = 10
T = 100
correlation_List = [0.4]

In [48]:
import numpy as np
from sklearn.model_selection import train_test_split

# Paramètres pour la simulation
num_samples = 16290  # Nombre d'échantillons à générer
L = 10  # Nombre de capteurs
T = 100  # Nombre de points temporels
phi_max = 90  # φmax
rho = 1  # Résolution
G = np.arange(-phi_max, phi_max + rho, rho)


In [49]:
import numpy as np
from numpy.linalg import eigh

def generate_music_spectrum(R, L, G, d=0.5, fs=1):
    """
    Calcule le spectre MUSIC à partir de la matrice de covariance R.

    :param R: Matrice de covariance estimée.
    :param L: Nombre de capteurs.
    :param G: Grille des angles (en degrés).
    :param d: Distance entre les éléments du réseau de capteurs (en longueurs d'onde).
    :param fs: Fréquence d'échantillonnage.
    :return: Spectre MUSIC pour les angles dans G.
    """
    # Calcul des valeurs propres et vecteurs propres
    eigvals, eigvecs = eigh(R)
    
    # Séparation des vecteurs du signal et du bruit
    E_noise = eigvecs[:, :L-2]  # Supposant 2 sources max
    
    # Calcul du spectre MUSIC
    P_music = np.zeros(len(G))
    for i, theta in enumerate(G):
        a = np.exp(np.arange(L) * 1j * 2 * np.pi * d * np.sin(np.radians(theta)) / fs)
        P_music[i] = 1 / np.abs(a.conj().T @ E_noise @ E_noise.conj().T @ a)
    
    # Normalisation du spectre
    P_music = P_music / np.max(P_music)
    
    return P_music

# Initialisation des tableaux pour les données
all_R_hat_with_phase = np.zeros((num_samples, L, L, 3))


for i in range(num_samples):
    # Générer les angles d'arrivée et la matrice X comme décrit précédemment
    thetaList = [np.random.uniform(-90, 90) for _ in range(2)]  # Exemple de génération d'angles
    varList = [np.random.uniform(0, 1000000000000) for _ in range(nbSources)]
    X = generate_X_matrix(nbSources, L, T, thetaList, varList, correlation_List, signal_noise_ratio)  # Votre fonction pour générer X, ajoutez les paramètres nécessaires
    
    # Calculer R_hat étendue avec phase
    R_hat_with_phase = generate_R_hat_with_phase(X)
    all_R_hat_with_phase[i] = R_hat_with_phase
    


# Utilisation de la fonction generate_music_spectrum sur vos données
all_music_spectra = np.zeros((num_samples, len(G)))
for i in range(num_samples):
    R = all_R_hat_with_phase[i, :, :, 0] + 1j * all_R_hat_with_phase[i, :, :, 1]  # Reconstitution de la matrice complexe
    all_music_spectra[i] = generate_music_spectrum(R, L, G)

# Division des données en ensembles d'entraînement et de test
X_train, X_test, Y_train, Y_test = train_test_split(all_R_hat_with_phase, all_music_spectra, test_size=0.2, random_state=42)

[[2.66957720e+10 5.36038177e+10]
 [5.36038177e+10 6.72711693e+11]]
[[6.21896973e+11 8.65085890e+10]
 [8.65085890e+10 7.52107695e+10]]
[[1.28625804e+10 3.62162820e+10]
 [3.62162820e+10 6.37323072e+11]]
[[9.09707297e+11 2.88643704e+11]
 [2.88643704e+11 5.72403813e+11]]
[[6.70322107e+11 2.60760637e+11]
 [2.60760637e+11 6.33987275e+11]]
[[6.50750744e+11 1.90646899e+11]
 [1.90646899e+11 3.49079894e+11]]
[[3.13253415e+11 1.24362080e+11]
 [1.24362080e+11 3.08574587e+11]]
[[4.72144952e+11 2.60804402e+11]
 [2.60804402e+11 9.00397958e+11]]
[[6.09068671e+11 1.81575511e+11]
 [1.81575511e+11 3.38320493e+11]]
[[3.76932752e+11 1.70144583e+11]
 [1.70144583e+11 4.80012335e+11]]
[[8.53504727e+11 3.01256459e+11]
 [3.01256459e+11 6.64579315e+11]]
[[7.40151501e+11 2.32585078e+11]
 [2.32585078e+11 4.56796840e+11]]
[[3.35962977e+11 9.78896293e+10]
 [9.78896293e+10 1.78263309e+11]]
[[4.46545084e+11 2.27136144e+11]
 [2.27136144e+11 7.22083132e+11]]
[[6.54920503e+11 2.44196300e+11]
 [2.44196300e+11 5.69075107e+

In [50]:
class DOADataset(Dataset):
    def __init__(self, covariance_matrices, spectra):
        """
        Initialisation du Dataset DOA.

        Args:
        - covariance_matrices (np.ndarray): Matrices de covariance étendues avec la phase.
          Forme attendue: (nb_samples, nbSensors, nbSensors, 3) pour les parties réelle, imaginaire, et la phase.
        - spectra (np.ndarray): Spectres MUSIC sous forme de vecteurs.
          Forme attendue: (nb_samples, len(G)), où len(G) est le nombre de points dans la grille DOA.
        """
        self.covariance_matrices = covariance_matrices
        self.spectra = spectra

    def __len__(self):
        return len(self.covariance_matrices)

    def __getitem__(self, idx):
        cov_matrix = self.covariance_matrices[idx]
        spectrum = self.spectra[idx]

        # Conversion en tenseur PyTorch
        cov_matrix = torch.tensor(cov_matrix, dtype=torch.float32)
        spectrum = torch.tensor(spectrum, dtype=torch.float32)

        return cov_matrix, spectrum

# Utilisation avec vos données X_train et Y_train
train_dataset = DOADataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

import torch.nn as nn

class DeepMusicModel(nn.Module):
    def __init__(self, num_classes=181):
        super(DeepMusicModel, self).__init__()

        # Couches convolutionnelles
        self.conv1 = nn.Conv2d(10, 256, kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv13 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        # Couches d'activation ReLU
        self.relu2 = nn.ReLU()
        self.relu5 = nn.ReLU()
        self.relu8 = nn.ReLU()
        self.relu11 = nn.ReLU()

        # Couches de normalisation
        self.norm3 = nn.BatchNorm2d(256)
        self.norm6 = nn.BatchNorm2d(256)
        self.norm9 = nn.BatchNorm2d(256)
        self.norm12 = nn.BatchNorm2d(256)

        # Couche de convolution (kernel 1x1)
        self.conv15 = nn.Conv2d(256, 256, kernel_size=1)

        # Couche de dropout
        self.dropout16 = nn.Dropout(0.5)

        # Couches entièrement connectées (FC)
        self.fc17 = nn.Linear(in_features=7680, out_features=num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu2(x)
        x = self.norm3(x)
        
        x = self.conv4(x)
        x = self.relu5(x)
        x = self.norm6(x)
        
        x = self.conv7(x)
        x = self.relu8(x)
        x = self.norm9(x)
        
        x = self.conv10(x)
        x = self.relu11(x)
        x = self.norm12(x)
        
        x = self.conv13(x)
        x = self.conv15(x)
        
        x = self.dropout16(x)
        x = x.view(x.size(0), -1)  # Aplatir pour FC
        x = self.fc17(x)
        

        return x


# Création de l'instance du modèle
model = DeepMusicModel()

# Définition de l'optimiseur et de la fonction de perte
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Préparation des données
dataset = DOADataset(X_train, Y_train)  # Supposer que X_train, Y_train sont définis
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

num_epochs = 10

# Boucle d'entraînement
for epoch in range(num_epochs):  # Définir num_epochs
    for cov_matrices, doa_labels in dataloader:
        optimizer.zero_grad()
        outputs = model(cov_matrices)
        loss = criterion(outputs, doa_labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

NameError: name 'generate_correlation_list' is not defined